In [88]:
import socket
import struct
import time

IP = "127.0.0.1"
PORT_SERVER = 27015
PORT_CLIENT = 27016

sock_send = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock_recv = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock_recv.bind((IP, PORT_CLIENT))

def inofs_send(s: str="", *args):
	b = b""
	if (len(args) > 0):
		b = struct.pack("<" + s[0]*len(args), *args)
	sock_send.sendto(bytes(s, "ascii") + b, (IP, PORT_SERVER))
    
def inofs_recv(format: str):
	data, _ = sock_recv.recvfrom(1024)
	return struct.unpack(format, data)

In [59]:
# Set up throttle control
inofs_send("dC;310A:c;089A:s;")
for i in range(11):
	inputs = 8
	throttle = 16384/10*i
	packet = struct.pack("<dd", inputs, throttle)
	inofs_send(b=packet)
	time.sleep(0.1);
inofs_send("dC;")

# Set up TAS speed monitor
inofs_send("fM;2B8:i;")
for i in range(10):
	values = inofs_recv("<ff")
	print("TAS: " + str(values[1]/128) + " knots")
inofs_send("fM;")

TAS: 1.0234375 knots
TAS: 1.0234375 knots
TAS: 1.0234375 knots
TAS: 1.0234375 knots
TAS: 1.0234375 knots
TAS: 1.0234375 knots
TAS: 1.0234375 knots
TAS: 1.0234375 knots
TAS: 1.0234375 knots
TAS: 1.0234375 knots


In [60]:
# Read if the sim is paused
inofs_send("fR:1;264:us;")
values = inofs_recv("<ff")
print(str(values[0]))

1.0


In [91]:
# Set game pause
pause = True
inofs_send("dW;262:us;", pause)